# Tensorflow の分散学習を SageMaker で行う

#### ノートブックに含まれる内容

- Tensorflow の分散学習を SageMaker で行うやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: MNIST
- データ: CNN

## セットアップ

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## データのロード

Tensorflow 向けに，protobuf フォーマットの TFRecord に変換して，変換済データを S3 にアップロードします．SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，MNIST データをダウンロードしてからローカルにある `utils.py` 変換処理を行って，S3 にアップロードします．

デフォルトでは SageMaker は sagemaker-{region}-{your aws account number} というバケットを使用します．当該バケットがない場合には，自動で新しく作成します．upload_data() メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することが可能です．

In [ ]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

以下を実行する前に，**<span style="color: red;">`data/mnist/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/mnist/XX')

## 分散学習を行う場合の，モデル記述の変更点

分散学習では，[グローバルステップ](https://www.tensorflow.org/api_docs/python/tf/train/global_step) と呼ばれる，インスタンス間で共有されるグローバル変数を使用します．そのため，オプティマイザの初期化時の引数に，以下のような記述でグローバル変数を引き渡します．

```python
train_op = optimizer.minimize(loss, tf.train.get_or_create_global_step())
```

これだけで，あとはインスタンスタイプを複数にすれば分散学習を行います．実行する前に，実行ファイルの中身を確認してみましょう．50 行目と 69 行目のところで，上記で示したグローバルステップの記述をしていることが確認できます．スクリプトは [TensorFlow MNIST example](https://github.com/tensorflow/models/tree/master/official/mnist) を使用しています．

In [ ]:
!cat 'mnist.py'

## モデルの学習を実行

学習プロセスは，通常の SageMaker での Tensorflow の使い方と変わりはありません．1 点だけ，分散学習を行うために `train_instance_count` が 2 になっていることを確認してください．

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             training_steps=10, 
                             evaluation_steps=3,
                             train_instance_count=2,
                             train_instance_type='ml.m4.xlarge')

mnist_estimator.fit(inputs)

# モデルの推論を実行


推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．モデルのデプロイには 10 分程度時間がかかります．

In [ ]:
mnist_predictor = mnist_estimator.deploy(initial_instance_count=1,
                                             instance_type='ml.m4.xlarge')

実際にエンドポイントを叩いてみましょう．

In [ ]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i].tolist()
    tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)
    predict_response = mnist_predictor.predict(tensor_proto)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response['outputs']['classes']['int64Val'][0]
    print("prediction is {}".format(prediction))

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker.Session().delete_endpoint(mnist_predictor.endpoint)